In [ ]:


# ============================================================================
# CONCLUSIONS
# ============================================================================


def afficher_conclusions(results):
    print("\n" + "=" * 70)
    print("PARTIE 10: CONCLUSIONS ET ANALYSES")
    print("=" * 70)

    print("\n📊 RÉSULTATS PRINCIPAUX:\n")

    # 1. Comparaison Otto vs Diesel (Gaz Parfait)
    print("1. COMPARAISON OTTO vs DIESEL (Gaz Parfait)")
    print("-" * 50)
    otto_gp_eta = results['Otto GP']['numerical_eff'] * 100
    diesel_gp_eta = results['Diesel GP']['numerical_eff'] * 100
    print(f"   Otto (τ=8):    η = {otto_gp_eta:.2f}%")
    print(f"   Diesel (τ=18): η = {diesel_gp_eta:.2f}%")
    print(f"   → Le Diesel est plus efficace de {diesel_gp_eta - otto_gp_eta:.2f} points")

    # 2. Impact de Van der Waals
    print("\n2. IMPACT DE VAN DER WAALS")
    print("-" * 50)
    otto_vdw_eta = results['Otto VdW']['numerical_eff'] * 100
    diesel_vdw_eta = results['Diesel VdW']['numerical_eff'] * 100
    print(f"   Otto:   GP={otto_gp_eta:.2f}%,   VdW={otto_vdw_eta:.2f}%   (Δ={otto_vdw_eta - otto_gp_eta:.2f}%)")
    print(f"   Diesel: GP={diesel_gp_eta:.2f}%, VdW={diesel_vdw_eta:.2f}% (Δ={diesel_vdw_eta - diesel_gp_eta:.2f}%)")
    print(f"   → VdW augmente légèrement le rendement")

    # 3. Validation numérique
    print("\n3. VALIDATION NUMÉRIQUE")
    print("-" * 50)
    for name, res in results.items():
        print(f"   {name:<12}: Erreur η_num vs η_th = {res['error_percent']:.4f}%")
    print("   → Excellente précision numérique (<0.01%)")

    # 4. Températures et pressions maximales
    print("\n4. TEMPÉRATURES ET PRESSIONS MAXIMALES")
    print("-" * 50)
    for name, res in results.items():
        print(f"   {name:<12}: T_max = {res['T_max']:.0f} K,  P_max = {res['P_max'] / 1e5:.0f} bar")

    # 5. Observations physiques
    print("\n5. OBSERVATIONS PHYSIQUES")
    print("-" * 50)
    print("   ✓ Le cycle Diesel a un meilleur rendement grâce à τ plus élevé")
    print("   ✓ Van der Waals prédit des pressions plus élevées")
    print("   ✓ L'écart GP-VdW augmente avec la compression")
    print("   ✓ Les formules théoriques sont validées numériquement")

    # 6. Perspectives d'amélioration
    print("\n6. PERSPECTIVES D'AMÉLIORATION")
    print("-" * 50)
    print("   • Ajouter les pertes réelles (frottements, transferts thermiques)")
    print("   • Modéliser la combustion réelle (cinétique chimique)")
    print("   • Étudier d'autres cycles (Stirling, Brayton)")
    print("   • Optimiser les paramètres (τ, α, γ)")


In [ ]:

# ============================================================================
# CONFIGURATION GLOBALE ET EXÉCUTION AUTOMATIQUE
# ============================================================================

# PARAMÈTRES À MODIFIER
PARAMS = {
    # Paramètres du gaz
    'n_moles': 1.0,
    'gamma': 1.4,
    'a_vdw': 0.1358,
    'b_vdw': 3.183e-5,

    # Paramètres cycle Otto
    'tau_otto': 8.0,
    'V_A_otto': 0.005,
    'T_A_otto': 300,
    'T_combustion_factor': 3.5,

    # Paramètres cycle Diesel
    'tau_diesel': 18.0,
    'alpha_diesel': 2.0,
    'V_1_diesel': 0.006,
    'T_1_diesel': 300,

    # Options d'affichage
    'afficher_graphiques': True,
}

print("=" * 80)
print(" 🚀 SIMULATION COMPLÈTE DES CYCLES THERMODYNAMIQUES ".center(80))
print("=" * 80)
print("\n✅ Configuration chargée !")
print(f"   • Gaz: n={PARAMS['n_moles']} mol, γ={PARAMS['gamma']}")
print(f"   • Cycle Otto: τ={PARAMS['tau_otto']}, V={PARAMS['V_A_otto'] * 1000}L, T={PARAMS['T_A_otto']}K")
print(
    f"   • Cycle Diesel: τ={PARAMS['tau_diesel']}, α={PARAMS['alpha_diesel']}, V={PARAMS['V_1_diesel'] * 1000}L, T={PARAMS['T_1_diesel']}K")

# ============================================================================
# ÉTAPE 1 : CRÉATION DES MODÈLES DE GAZ
# ============================================================================

print("\n" + "=" * 80)
print(" ÉTAPE 1 : CRÉATION DES MODÈLES DE GAZ ".center(80))
print("=" * 80)

# Créer les modèles de gaz
gaz_parfait = Gaz_Parfait(n=PARAMS['n_moles'], gamma=PARAMS['gamma'])
gaz_vdw = Gaz_Reel(
    n=PARAMS['n_moles'],
    gamma=PARAMS['gamma'],
    a=PARAMS['a_vdw'],
    b=PARAMS['b_vdw']
)

print(f"\n✓ Gaz Parfait créé : {gaz_parfait}")
print(f"✓ Gaz Van der Waals créé : {gaz_vdw}")


# Test rapide
def tester_modeles_gaz(n: float, gamma: float, a: float, b: float, V_test: float, T_test: float):
    """Teste les modèles Gaz_Parfait et Gaz_Reel"""
    print("\n--- Test des modèles de gaz ---")

    gas_ideal = Gaz_Parfait(n=n, gamma=gamma)
    gas_vdw = Gaz_Reel(n=n, gamma=gamma, a=a, b=b)

    print(f"\n1. {gas_ideal}")
    P_ideal = gas_ideal.pression(V_test, T_test)
    print(f"   P(V={V_test}, T={T_test}) = {P_ideal:.2f} Pa = {P_ideal / 1e5:.2f} bar")
    print(f"   Cv = {gas_ideal.cv():.2f} J/K")
    print(f"   Cp = {gas_ideal.cp():.2f} J/K")

    print(f"\n2. {gas_vdw}")
    P_vdw = gas_vdw.pression(V_test, T_test)
    print(f"   P(V={V_test}, T={T_test}) = {P_vdw:.2f} Pa = {P_vdw / 1e5:.2f} bar")

    ecart = abs(P_vdw - P_ideal) / P_ideal * 100
    print(f"   Écart avec GP: {ecart:.2f}%")

    if P_vdw < P_ideal:
        print("    → Le gaz réel prédit une pression plus faible que le gaz parfait.")
        print("      (Interactions attractives)")
    else:
        print("    → Le gaz réel prédit une pression plus élevée que le gaz parfait.")
        print("      (Volume propre des molécules)")


tester_modeles_gaz(n=3.0, gamma=1.3, a=0.2, b=4e-5, V_test=0.02, T_test=350)

# ============================================================================
# ÉTAPE 2 : CRÉATION DES CYCLES OTTO
# ============================================================================

print("\n" + "=" * 80)
print(" ÉTAPE 2 : CRÉATION DES CYCLES OTTO ".center(80))
print("=" * 80)

otto_gp = OttoCycle(
    gaz=gaz_parfait,
    tau=PARAMS['tau_otto'],
    V_A=PARAMS['V_A_otto'],
    T_A=PARAMS['T_A_otto'],
    T_combustion_factor=PARAMS['T_combustion_factor']
)

otto_vdw = OttoCycle(
    gaz=gaz_vdw,
    tau=PARAMS['tau_otto'],
    V_A=PARAMS['V_A_otto'],
    T_A=PARAMS['T_A_otto'],
    T_combustion_factor=PARAMS['T_combustion_factor']
)

print(f"\n✓ Cycle Otto (Gaz Parfait) créé")
print(f"✓ Cycle Otto (Van der Waals) créé")

res_otto_gp = EfficiencyCalculator.analyze_cycle(otto_gp, 'otto')
res_otto_vdw = EfficiencyCalculator.analyze_cycle(otto_vdw, 'otto')

print(f"\n📊 RÉSULTATS OTTO :")
print(f"   Gaz Parfait :")
print(f"      • Rendement : {res_otto_gp['numerical_eff'] * 100:.2f}%")
print(f"      • Travail   : {res_otto_gp['work']:.2f} J")
print(f"      • T_max     : {res_otto_gp['T_max']:.0f} K")
print(f"      • P_max     : {res_otto_gp['P_max'] / 1e5:.0f} bar")

print(f"\n   Van der Waals :")
print(f"      • Rendement : {res_otto_vdw['numerical_eff'] * 100:.2f}%")
print(f"      • Travail   : {res_otto_vdw['work']:.2f} J")
print(f"      • T_max     : {res_otto_vdw['T_max']:.0f} K")
print(f"      • P_max     : {res_otto_vdw['P_max'] / 1e5:.0f} bar")

ecart_otto = ((res_otto_vdw['numerical_eff'] - res_otto_gp['numerical_eff']) / res_otto_gp['numerical_eff']) * 100
print(f"\n   💡 Écart GP→VdW : {ecart_otto:+.2f}%")

# ============================================================================
# ÉTAPE 3 : CRÉATION DES CYCLES DIESEL
# ============================================================================

print("\n" + "=" * 80)
print(" ÉTAPE 3 : CRÉATION DES CYCLES DIESEL ".center(80))
print("=" * 80)

diesel_gp = DieselCycle(
    gaz=gaz_parfait,
    tau=PARAMS['tau_diesel'],
    alpha=PARAMS['alpha_diesel'],
    V_1=PARAMS['V_1_diesel'],
    T_1=PARAMS['T_1_diesel']
)

diesel_vdw = DieselCycle(
    gaz=gaz_vdw,
    tau=PARAMS['tau_diesel'],
    alpha=PARAMS['alpha_diesel'],
    V_1=PARAMS['V_1_diesel'],
    T_1=PARAMS['T_1_diesel']
)

print(f"\n✓ Cycle Diesel (Gaz Parfait) créé")
print(f"✓ Cycle Diesel (Van der Waals) créé")

res_diesel_gp = EfficiencyCalculator.analyze_cycle(diesel_gp, 'diesel')
res_diesel_vdw = EfficiencyCalculator.analyze_cycle(diesel_vdw, 'diesel')

print(f"\n📊 RÉSULTATS DIESEL :")
print(f"   Gaz Parfait :")
print(f"      • Rendement : {res_diesel_gp['numerical_eff'] * 100:.2f}%")
print(f"      • Travail   : {res_diesel_gp['work']:.2f} J")
print(f"      • T_max     : {res_diesel_gp['T_max']:.0f} K")
print(f"      • P_max     : {res_diesel_gp['P_max'] / 1e5:.0f} bar")

print(f"\n   Van der Waals :")
print(f"      • Rendement : {res_diesel_vdw['numerical_eff'] * 100:.2f}%")
print(f"      • Travail   : {res_diesel_vdw['work']:.2f} J")
print(f"      • T_max     : {res_diesel_vdw['T_max']:.0f} K")
print(f"      • P_max     : {res_diesel_vdw['P_max'] / 1e5:.0f} bar")

ecart_diesel = ((res_diesel_vdw['numerical_eff'] - res_diesel_gp['numerical_eff']) / res_diesel_gp[
    'numerical_eff']) * 100
print(f"\n   💡 Écart GP→VdW : {ecart_diesel:+.2f}%")

# ============================================================================
# ÉTAPE 4 : TABLEAU RÉCAPITULATIF
# ============================================================================

print("\n" + "=" * 80)
print(" ÉTAPE 4 : TABLEAU RÉCAPITULATIF ".center(80))
print("=" * 80)

resultats = {
    'Otto GP': res_otto_gp,
    'Otto VdW': res_otto_vdw,
    'Diesel GP': res_diesel_gp,
    'Diesel VdW': res_diesel_vdw
}

print(f"\n{'Configuration':<15} {'η (%)':<10} {'W (J)':<10} {'T_max (K)':<12} {'P_max (bar)':<12}")
print("-" * 80)
for nom, res in resultats.items():
    print(
        f"{nom:<15} {res['numerical_eff'] * 100:>8.2f} {res['work']:>9.2f} {res['T_max']:>11.0f} {res['P_max'] / 1e5:>11.0f}")
print("-" * 80)

meilleur = max(resultats.items(), key=lambda x: x[1]['numerical_eff'])
print(f"\n🏆 Meilleur cycle : {meilleur[0]} avec η = {meilleur[1]['numerical_eff'] * 100:.2f}%")

# ============================================================================
# ÉTAPE 5 : VISUALISATIONS
# ============================================================================

if PARAMS['afficher_graphiques']:
    print("\n" + "=" * 80)
    print(" ÉTAPE 5 : VISUALISATIONS ".center(80))
    print("=" * 80)

    plotter = CyclePlotter()

    print("\n📈 1. Comparaison Otto : Gaz Parfait vs Van der Waals")
    plotter.plot_comparison(
        [otto_gp, otto_vdw],
        ['Otto - Gaz Parfait', 'Otto - Van der Waals'],
        ['blue', 'red'],
        title=f'Cycle Otto (τ={PARAMS["tau_otto"]}) : Gaz Parfait vs Van der Waals'
    )

    print("\n📈 2. Comparaison Diesel : Gaz Parfait vs Van der Waals")
    plotter.plot_comparison(
        [diesel_gp, diesel_vdw],
        ['Diesel - Gaz Parfait', 'Diesel - Van der Waals'],
        ['green', 'orange'],
        title=f'Cycle Diesel (τ={PARAMS["tau_diesel"]}, α={PARAMS["alpha_diesel"]}) : GP vs VdW'
    )

    print("\n📈 3. Comparaison Otto vs Diesel (Gaz Parfait)")
    plotter.plot_comparison(
        [otto_gp, diesel_gp],
        [f'Otto (τ={PARAMS["tau_otto"]})', f'Diesel (τ={PARAMS["tau_diesel"]}, α={PARAMS["alpha_diesel"]})'],
        ['blue', 'green'],
        title='Comparaison Otto vs Diesel - Gaz Parfait'
    )

    print("\n📈 4. Comparaison complète (4 configurations)")
    plotter.plot_comparison(
        [otto_gp, otto_vdw, diesel_gp, diesel_vdw],
        ['Otto GP', 'Otto VdW', 'Diesel GP', 'Diesel VdW'],
        ['blue', 'red', 'green', 'orange'],
        title='Comparaison complète des 4 configurations'
    )

    print("\n📈 5. Diagramme des rendements")
    plotter.plot_efficiency_comparison(resultats)

# ============================================================================
# ÉTAPE 6 : DÉTAILS DES ÉTATS THERMODYNAMIQUES
# ============================================================================

print("\n" + "=" * 80)
print(" ÉTAPE 6 : DÉTAILS DES ÉTATS THERMODYNAMIQUES ".center(80))
print("=" * 80)

print("\n🔍 CYCLE OTTO (Gaz Parfait) :")
print("-" * 80)
for nom, (P, V, T) in otto_gp.etat.items():
    print(f"  Point {nom}: P={P / 1e5:>7.2f} bar, V={V * 1000:>7.2f} L, T={T:>7.0f} K")

print("\n🔍 CYCLE DIESEL (Gaz Parfait) :")
print("-" * 80)
for nom, (P, V, T) in diesel_gp.etat.items():
    print(f"  Point {nom}: P={P / 1e5:>7.2f} bar, V={V * 1000:>7.2f} L, T={T:>7.0f} K")

# ============================================================================
# ÉTAPE 7 : ANALYSE ÉNERGÉTIQUE PAR TRANSFORMATION
# ============================================================================

print("\n" + "=" * 80)
print(" ÉTAPE 7 : ANALYSE ÉNERGÉTIQUE - CYCLE OTTO (GP) ".center(80))
print("=" * 80)

for nom_chemin, (V, P, T) in otto_gp.chemin.items():
    W_chemin = trapezoid(P, V)
    T_debut = T[0]
    T_fin = T[-1]

    V_var = abs(V[-1] - V[0])
    P_var = abs(P[-1] - P[0])

    if V_var < 1e-6:
        type_transfo = "ISOCHORE (V=cste)"
        Q_chemin = otto_gp.gaz.cv() * (T_fin - T_debut)
    elif P_var < 1e3:
        type_transfo = "ISOBARE (P=cste)"
        Q_chemin = otto_gp.gaz.cp() * (T_fin - T_debut)
    else:
        type_transfo = "ADIABATIQUE (Q=0)"
        Q_chemin = 0.0

    print(f"\n🔄 Transformation {nom_chemin} - {type_transfo}")
    print(f"    Travail (W)  : {W_chemin:>8.2f} J")
    print(f"    Chaleur (Q)  : {Q_chemin:>8.2f} J")
    print(f"    ΔT           : {T_fin - T_debut:>8.2f} K")

# ============================================================================
# PARTIE 9: ÉTUDE PARAMÉTRIQUE
# ============================================================================

res_param9 = etude_parametrique(
    gas_ideal=gaz_parfait,
    gas_vdw=gaz_vdw,
    V_OTTO=PARAMS['V_A_otto'],
    V_DIESEL=PARAMS['V_1_diesel'],
    T_INITIAL=PARAMS['T_A_otto']
)

# ============================================================================
# RÉSUMÉ FINAL
# ============================================================================

print("\n" + "=" * 80)
print(" 🎉 SIMULATION TERMINÉE AVEC SUCCÈS ! ".center(80))
print("=" * 80)

print("\n📊 RÉSUMÉ FINAL :")
print("-" * 80)

print(f"\n1. RENDEMENTS :")
print(f"   • Otto (GP)    : {res_otto_gp['numerical_eff'] * 100:.2f}%")
print(f"   • Otto (VdW)   : {res_otto_vdw['numerical_eff'] * 100:.2f}%")
print(f"   • Diesel (GP)  : {res_diesel_gp['numerical_eff'] * 100:.2f}%")
print(f"   • Diesel (VdW) : {res_diesel_vdw['numerical_eff'] * 100:.2f}%")

print(f"\n2. COMPARAISON OTTO vs DIESEL (Gaz Parfait) :")
diff_rendement = res_diesel_gp['numerical_eff'] - res_otto_gp['numerical_eff']
print(f"   • Le Diesel est plus efficace de {diff_rendement * 100:.2f} points de %")
print(f"   • Raison : τ_Diesel ({PARAMS['tau_diesel']}) > τ_Otto ({PARAMS['tau_otto']})")

print(f"\n3. IMPACT VAN DER WAALS :")
print(f"   • Otto   : {ecart_otto:+.2f}%")
print(f"   • Diesel : {ecart_diesel:+.2f}%")
if abs(ecart_otto) < 1 and abs(ecart_diesel) < 1:
    print(f"   → Écarts faibles : modèle gaz parfait très précis")
else:
    print(f"   → Écarts significatifs : effets VdW importants")

print(f"\n4. CONDITIONS EXTRÊMES :")
print(f"   • T_max Otto    : {res_otto_gp['T_max']:.0f} K ({res_otto_gp['T_max'] - 273:.0f}°C)")
print(f"   • T_max Diesel  : {res_diesel_gp['T_max']:.0f} K ({res_diesel_gp['T_max'] - 273:.0f}°C)")
print(f"   • P_max Otto    : {res_otto_gp['P_max'] / 1e5:.0f} bar")
print(f"   • P_max Diesel  : {res_diesel_gp['P_max'] / 1e5:.0f} bar")

print("\n✅ Tous les objectifs atteints :")
print("   ✓ Modélisation POO complète")
print("   ✓ Cycles Otto et Diesel implémentés")
print("   ✓ Gaz Parfait et Van der Waals comparés")
print("   ✓ Calculs numériques validés")
print("   ✓ Visualisations générées")
print("   ✓ Analyses énergétiques réalisées")

print("\n" + "=" * 80)

# ============================================================================
# PARTIE 10: CONCLUSIONS
# ============================================================================

afficher_conclusions(resultats)